# **Import Required Packages**

In [5]:
#check if GPU is available
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


In [13]:
#clone yolov5 github repo
!git clone https://github.com/ultralytics/yolov5

Cloning into 'yolov5'...
remote: Enumerating objects: 17483, done.
remote: Counting objects: 100% (108/108), done.
remote: Compressing objects: 100% (78/78), done.
remote: Total 17483 (delta 79), reused 30 (delta 30), pack-reused 17375 (from 3)
Receiving objects: 100% (17483/17483), 16.33 MiB | 22.33 MiB/s, done.
Resolving deltas: 100% (11983/11983), done.


In [14]:
%cd yolov5

/content/yolov5/yolov5


In [8]:
!pip install -r requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 92.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 68.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 37.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 34.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 47.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling 

# **Load Dataset**

In [15]:
#unzip the uploaded trainval.zip file
!unzip /content/yolov5/trainval.zip -d /content/yolov5/data

Streaming output truncated to the last 5000 lines.
  inflating: /content/yolov5/data/trainval/Annotations/img021939.xml  
  inflating: /content/yolov5/data/trainval/Annotations/img021940.xml  
  inflating: /content/yolov5/data/trainval/Annotations/img021941.xml  
  inflating: /content/yolov5/data/trainval/Annotations/img021942.xml  
  inflating: /content/yolov5/data/trainval/Annotations/img021943.xml  
  inflating: /content/yolov5/data/trainval/Annotations/img021944.xml  
  inflating: /content/yolov5/data/trainval/Annotations/img021945.xml  
  inflating: /content/yolov5/data/trainval/Annotations/img021946.xml  
  inflating: /content/yolov5/data/trainval/Annotations/img021947.xml  
  inflating: /content/yolov5/data/trainval/Annotations/img021948.xml  
  inflating: /content/yolov5/data/trainval/Annotations/img021949.xml  
  inflating: /content/yolov5/data/trainval/Annotations/img021950.xml  
  inflating: /content/yolov5/data/trainval/Annotations/img021951.xml  
  inflating: /content/yolo

# ***Convert XML to TXT***

In [31]:
import os
import xml.etree.ElementTree as ET
#Defining Class of Dataset
class_mapping = {
    "crack": 0,
    "finger" : 1,
    "black_core" : 2,
    "thick_line" : 3,
    "star_crack" : 4,
    "corner" : 5,
    "fragment" : 6,
    "scratch" : 7,
    "horizontal_dislocation" : 8,
    "vertical_dislocation" : 9,
    "rinting_error" : 10,
    "short_circuit" : 11
}
#paths
xml_dir = "/content/yolov5/data/trainval/Annotations"
img_dir = "/content/yolov5/data/trainval/JPEGImages"
output_img_dir = "/content/yolov5/data/trainval/Images/train"
output_label_dir = "/content/yolov5/data/trainval/Labels/train"

os.makedirs(output_img_dir, exist_ok=True)
os.makedirs(output_label_dir, exist_ok=True)

# Convert each XML
for xml_file in os.listdir(xml_dir):
    if not xml_file.endswith('.xml'):
        continue

    tree = ET.parse(os.path.join(xml_dir, xml_file))
    root = tree.getroot()
    img_filename = root.find('filename').text
    img_path = os.path.join(img_dir, img_filename)

    # Copy image
    if os.path.exists(img_path):
        os.system(f"cp '{img_path}' '{output_img_dir}'")

    size = root.find('size')
    img_w = int(size.find('width').text)
    img_h = int(size.find('height').text)

    label_path = os.path.join(output_label_dir, xml_file.replace('.xml', '.txt'))
    with open(label_path, 'w') as f:
        for obj in root.findall('object'):
            cls = obj.find('name').text.lower()
            if cls not in class_mapping:
                continue
            cls_id = class_mapping[cls]

            bbox = obj.find('bndbox')
            xmin = int(bbox.find('xmin').text)
            ymin = int(bbox.find('ymin').text)
            xmax = int(bbox.find('xmax').text)
            ymax = int(bbox.find('ymax').text)

            x_center = ((xmin + xmax) / 2) / img_w
            y_center = ((ymin + ymax) / 2) / img_h
            width = (xmax - xmin) / img_w
            height = (ymax - ymin) / img_h

            f.write(f"{cls_id} {x_center} {y_center} {width} {height}\n")

# **Spliting into Training and Testing Dataset**

In [32]:
import os
import shutil
import random

#Defining Paths
img_train_dir = '/content/yolov5/data/trainval/Images/train'
label_train_dir = '/content/yolov5/data/trainval/Labels/train'
img_val_dir = '/content/yolov5/data/trainval/Images/val'
label_val_dir = '/content/yolov5/data/trainval/Labels/val'

#Create val folder
os.makedirs(img_val_dir, exist_ok=True)
os.makedirs(label_val_dir, exist_ok=True)

# Get all image filenames
img_files = [f for f in os.listdir(img_train_dir) if f.endswith('.jpg') or f.endswith('.png')]
random.seed(42)
random.shuffle(img_files)

#Splitting 80% training & 20% validation
val_count = int(0.2* len(img_files))
val_files = img_files[:val_count]
for file in val_files:
  base_name = os.path.splitext(file)[0]

  #Move Image
  shutil.move(os.path.join(img_train_dir, file), os.path.join(img_val_dir, file))

  #Move Label
  label_file = f"{base_name}.txt"
  if os.path.exists(os.path.join(label_train_dir, label_file)):
    shutil.move(os.path.join(label_train_dir, label_file), os.path.join(label_val_dir, label_file))

# **Creating YAML File**

In [54]:
data_yaml = """
path: /content/dataset
train: Images/train
val: Images/val
nc: 12
names: ['crack', 'finger', 'black_core', 'thick_line', 'star_crack', 'corner', 'fragment', 'scratch',
'horizontal_dislocation', 'vertical_dislocation', 'printing_error', 'short_circuit']
"""

with open('/content/dataset/pvel_ad.yaml', 'w') as f:
    f.write(data_yaml)


# **Creating hyperparameter YAML file for Data Augmentation**

In [44]:
# Save hyp.yaml in yolov5/data/
custom_hyp = """
# Learning parameters
lr0: 0.01          # Initial learning rate
lrf: 0.1           # Final learning rate (multiplier)
momentum: 0.937    # SGD momentum
weight_decay: 0.0005

# Augmentation parameters
hsv_h: 0.015       # Image HSV-Hue augmentation (0–0.1)
hsv_s: 0.7         # Image HSV-Saturation augmentation (0–1)
hsv_v: 0.4         # Image HSV-Value augmentation (0–1)
degrees: 0.0       # Image rotation (+/- deg)
translate: 0.1     # Image translation (+/- fraction)
scale: 0.5         # Image scale (+/- gain)
shear: 0.0         # Image shear (+/- deg)
perspective: 0.0   # Perspective transformation
flipud: 0.0        # Flip up-down probability
fliplr: 0.5        # Flip left-right probability
mosaic: 1.0        # Mosaic augmentation probability
mixup: 0.2         # MixUp augmentation probability
copy_paste: 0.0    # Copy-Paste augmentation (only works for segmentation)

# Additional training hyperparameters
warmup_epochs: 3.0
warmup_momentum: 0.8
warmup_bias_lr: 0.1
box: 0.05          # Box loss gain
cls: 0.5           # Class loss gain
cls_pw: 1.0        # Class loss positive weight
obj: 1.0           # Objectness loss gain (scale with pixels)
obj_pw: 1.0        # Objectness loss positive weight
anchor_t: 4.0      # Anchor-multiple threshold
anchors: 3         # Number of anchors

"""

with open('/content/yolov5/data/trainval/hyperparameter.yaml', 'w') as f:
    f.write(custom_hyp)


# **Train the YOLOv5 Model**

In [45]:
#check the directory
%cd /content/yolov5/

/content/yolov5


In [50]:
#Start Training the Model
#!python train.py \
 # --img 640 \
 #  --batch 16 \
 # --epochs 100 \
  #--data data/trainval/pvel_ad.yaml \
  #--weights yolov5s.pt \
  #--name solar-defect-yolov5s \
  #--cache


wandb: WARNING ⚠️ wandb is deprecated and will be removed in a future release. See supported integrations at https://github.com/ultralytics/yolov5#integrations.
2025-05-21 11:30:11.696020: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747827011.730357   35757 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747827011.741302   35757 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice: (30 second timeout) 
wandb: W&B disabled due to login timeout.
train: weights=yolov5s.pt, cfg=, data=data/trainval/pvel_ad.yaml, hyp=data/hyp

In [24]:
import glob

train_labels = glob.glob('/content/yolov5/data/trainval/Images/train/*.jpg')
val_labels = glob.glob('/content/yolov5/data/trainval/Labels/train/*.txt')

print(f"Train labels found: {len(train_labels)}")
print(f"Val labels found: {len(val_labels)}")


Train labels found: 3600
Val labels found: 3600


In [47]:
import os

image_dir = '/content/yolov5/data/trainval/Images/train'
label_dir = '/content/yolov5/data/trainval/Labels/train'

image_files = [f for f in os.listdir(image_dir) if f.endswith(('.jpg', '.png'))]
label_files = [f for f in os.listdir(label_dir) if f.endswith('.txt')]

unmatched = [img for img in image_files if os.path.splitext(img)[0] + '.txt' not in label_files]

print(f"Total images: {len(image_files)}")
print(f"Total labels: {len(label_files)}")
print(f"Images without labels: {len(unmatched)}")


Total images: 3600
Total labels: 3600
Images without labels: 0


In [51]:
!ls /content/yolov5/data/trainval/Images/train | head
!ls /content/yolov5/data/trainval/Labels/train | head


img000001.jpg
img000002.jpg
img000004.jpg
img000005.jpg
img000006.jpg
img000007.jpg
img000008.jpg
img000009.jpg
img000010.jpg
img000011.jpg
img000001.txt
img000002.txt
img000004.txt
img000005.txt
img000006.txt
img000007.txt
img000008.txt
img000009.txt
img000010.txt
img000011.txt


In [53]:
!mv /content/yolov5/data/trainval /content/dataset

mv: cannot stat '/content/yolov5/data/trainval': No such file or directory


In [56]:
!python3 val.py --data /content/dataset/pvel_ad.yaml --weights yolov5s.pt --img 640 --batch 16


val: data=/content/dataset/pvel_ad.yaml, weights=['yolov5s.pt'], batch_size=16, imgsz=640, conf_thres=0.001, iou_thres=0.6, max_det=300, task=val, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=runs/val, name=exp, exist_ok=False, half=False, dnn=False
YOLOv5 🚀 v7.0-419-gcd44191c Python-3.11.12 torch-2.6.0+cu124 CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Traceback (most recent call last):
  File "/content/yolov5/val.py", line 604, in <module>
    main(opt)
  File "/content/yolov5/val.py", line 575, in main
    run(**vars(opt))
  File "/usr/local/lib/python3.11/dist-packages/torch/utils/_contextlib.py", line 116, in decorate_context
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^
  File "/content/yolov5/val.py", line 297, in run
    assert ncm == nc, (
           ^^^^^^^^^
AssertionError: ['yolov5s.pt'] (80 classes) trai

In [59]:
!python3 train.py \
  --img 640 \
  --batch 16 \
  --epochs 100 \
  --data /content/dataset/pvel_ad.yaml \
  --weights 12 \
  --name solar-defect-yolov5s \
  --cache


wandb: WARNING ⚠️ wandb is deprecated and will be removed in a future release. See supported integrations at https://github.com/ultralytics/yolov5#integrations.
2025-05-21 11:55:40.490327: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747828540.515808   42007 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747828540.523075   42007 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice: (30 second timeout) 
wandb: W&B disabled due to login timeout.
train: weights=12, cfg=, data=/content/dataset/pvel_ad.yaml, hyp=data/hyps/hyp